### Code

In [1]:
import numpy as np
from scipy.optimize import minimize


class LogisticRegression(object):
    
    default_tol = 1e-6  # solver tol
    default_penalty = 1.0  # coef of the regularization term
    _exp_lim = 40  # exp overflow range

    def __init__(self, tol=default_tol,
                 standardize=True,
                 regularization=None,
                 penalty=default_penalty,
                 random_state=None):
        self.tol = tol
        self.standardize = standardize
        self.penalty = penalty
        self.regularization = regularization
        if random_state is not None:
            np.random.seed(random_state)
        
    def fit(self, X, y):
        X_train = self.transform(X, fit=True)
        y_train = self.convert_y(y, how="to_binary")
        self.n_params = len(X_train[0])  # include intercept
        beta_hat = minimize(self.loss,
                            x0=np.zeros(self.n_params),
                            method='BFGS',
                            tol=self.tol,
                            args=(X_train, y_train),
                           )
        self.beta = beta_hat.x
    
    def predict(self, X):
        X_test = self.transform(X, fit=False)
        Xbeta = (X_test * self.beta).sum(axis=1)
        p = np.fromiter((1. / (1 + np.exp(-xb)) if xb < self._exp_lim and xb > - self._exp_lim
                         else 1 if xb >= self._exp_lim
                         else 0 for xb in Xbeta), dtype=float)
        y_test = (p >= 0.5).astype(int)
        return self.convert_y(y_test, how="from_binary")
        
    def loss(self, beta, X_train, y_train):
        beta = np.array(beta)
        Xbeta = (X_train * beta).sum(axis=1)
        loglikelihood_p1 = (Xbeta * y_train).sum()
        # loglikelihood_p2 = -np.log(1 + np.exp(Xbeta)).sum()
        # may need to approx to avoid overflow
        loglikelihood_p2 = -np.sum(np.fromiter((np.log(1 + np.exp(xb)) if xb < self._exp_lim and xb > - self._exp_lim / 2
                                               else xb if xb >= self._exp_lim
                                               else np.exp(xb)
                                               for xb in Xbeta), dtype=float))
        loss = - loglikelihood_p1 - loglikelihood_p2
        if self.regularization == 'l1':
            loss += self.penalty * np.sum(np.abs(beta))
        elif self.regularization == 'l2':
            loss += self.penalty * np.sum(beta ** 2)
        return loss
    
    def transform(self, X, fit=True):
        X = np.array(X)
        if self.standardize:
            if fit:
                self.X_mean = X.mean(axis=0)
                self.X_std = X.std(axis=0)
                if 0. in self.X_std:
                    raise ZeroDivisionError('Please remove the column with identical values')
            X = (X - self.X_mean) / self.X_std
        return np.hstack([np.ones(len(X)).reshape(len(X), 1), X])
        
        
    def convert_y(self, y, how='to_binary'):
        if how == 'to_binary':
            self.levels = list(set(y))
            return np.array([0 if yi == self.levels[0] else 1 for yi in y])
        elif how == 'from_binary':
            return np.array([self.levels[yi] for yi in y])
        else:
            raise ValueError('argument "how" received an invalid string')

### Comparison with sklearn.linear_model.LogisticRegression

#### prepare data

In [2]:
import pandas as pd
np.random.seed(271828)

df = pd.read_csv('test_data/iris.csv')
df['species'] = (df.species == 'setosa').astype(int)
df = df.sample(frac=1).reset_index(drop=True)
train_x, test_x = df[:len(df)//2], df[len(df)//2:]
train_y, test_y = train_x.pop('species'), test_x.pop('species')

#### our implementation

In [3]:
lr1 = LogisticRegression(regularization='l2', penalty=20, random_state=314159)
lr1.fit(train_x, train_y)
1. * (lr1.predict(test_x) == test_y).sum() / len(test_y), lr1.beta

(1.0, array([-0.28580334, -0.25264217,  0.26168977, -0.36024665, -0.33702623]))

#### sklearn result

In [4]:
from sklearn.linear_model import LogisticRegression as sklearnLogisticRegression

lr0 = sklearnLogisticRegression(fit_intercept=True, penalty='l2', C=1)  # not standardized
lr0.fit(train_x, train_y)
pred_y0 = lr0.predict(test_x)
1. * (lr0.predict(test_x) == test_y).sum() / len(test_y), lr0.intercept_, lr0.coef_

(1.0,
 array([6.39932254]),
 array([[-0.45979489,  0.64929543, -1.99067379, -0.76921295]]))